# Adiabatic Pipe Flow

Consider the following information on a well insulated steam pipe from measurement equipment:

| Parameter | Location | Value | Unit |
|:--------- |:-------- | -----:| ---- |
| Temperature | Inlet | 220 | °C |
| Pressure | Inlet | 10 | bar |
|  | Outlet | 9 | bar |

With the energy balance of the thermodynamic open system, where the work $\dot W$ and heat $\dot Q$ transferred change the enthalpy $h$ of a mass flow $\dot m$ from a state 1 to a different state 2, we can derive the missing outlet temperature:

```{math}
    :label: eq:pipe-flow-energy-balance
    \dot W + \dot Q = \dot m \cdot \left(h_2 - h_1\right)
```

Since a pipe does not transfer work to the fluid, and we are assuming a well insulated pipe, we can assume that both, work and heat transferred, are equal to 0. Therefore the change in enthalpy must be zero as well:

```{math}
    :label: eq:pipe-flow-energy-balance-simplified
    0 = \dot m \cdot \left(h_2 - h_1\right)\\
    h_2 = h_1
```

To get the temperature, we need to check for the outlet pressure and the ...

```{math}
    :label: eq:pipe-flow-outlet-temperature
    T_2 = T\left(p_2, h_1\right)\\
    h_1 = \left(T_1, p_1\right)
```



We have seen, that the change in enthalpy is zero, since no energy has been transferred. Therefore, one could say, no energy has been lost. However, by we cannot expect to find a process, that increases the pressure back to the inlet's level without doing anything. While we have not lost any energy to the ambient, the energy has obviously become less valuable. The Second Law Exergy analysis can make this change visible. We can calculate the exergy at the inlet and the outlet of the pipe and see how it has been affected by the pressure losses.

First, we create a function that calculates the physical exergy of a fluid:

In [8]:
from CoolProp.CoolProp import PropsSI as PSI


def calc_physical_exergy(p, T, p0, T0, fluid):
    r"""
    Calculate specific physical exergy.

    Physical exergy is allocated to a thermal and a mechanical share according
    to :cite:`Morosuk2019`.

    Parameters
    ----------
    p : float
        Pressure of fluid p / Pa.
        
    T : float
        Temperature of fluid T / K.

    p0 : float
        Ambient pressure p0 / Pa.

    T0 : float
        Ambient temperature T0 / K.

    Returns
    -------
    e_ph : tuple
        Specific thermal and mechanical exergy
        (:math:`e^\mathrm{T}`, :math:`e^\mathrm{M}`) in J / kg.

        .. math::

            e^\mathrm{T} = \left( h - h \left( p, T_0 \right) \right) -
            T_0 \cdot \left(s - s\left(p, T_0\right)\right)

            e^\mathrm{M}=\left(h\left(p,T_0\right)-h\left(p_0,T_0\right)\right)
            -T_0\cdot\left(s\left(p, T_0\right)-s\left(p_0,T_0\right)\right)

            e^\mathrm{PH} = e^\mathrm{T} + e^\mathrm{M}
    """
    h = PSI("H", "P", p, "T", T, fluid)
    s = PSI("S", "P", p, "T", T, fluid)
    
    h_T0_p = PSI("H", "P", p, "T", T0, fluid)
    s_T0_p = PSI("S", "P", p, "T", T0, fluid)
    
    ex_therm = (h - h_T0_p) - T0 * (s - s_T0_p)
    
    h0 = PSI("H", "P", p0, "T", T0, fluid)
    s0 = PSI("S", "P", p0, "T", T0, fluid)
    
    ex_mech = (h_T0_p - h0) - T0 * (s_T0_p - s0)
    return ex_therm, ex_mech

Then, we can calculate the exergy and the inlet and the outlet state.

In [21]:
fluid = "water"
p0 = 1.01325 * 1e5
T0 = 298.15
h0 = PSI("H", "P", p0, "T", T0, fluid)

p_in = 10 * 1e5
T_in = 220 + 273.15
h_in = PSI("H", "P", p_in, "T", T_in, fluid)

p_out = 6 * 1e5
h_out = h_in
T_out = PSI("T", "P", p_out, "H", h_out, fluid)

ex_in = calc_physical_exergy(p_in, T_in, p0, T0, fluid)
ex_out = calc_physical_exergy(p_out, T_out, p0, T0, fluid)

First, we can observe the change in total specific exergy:

In [22]:
sum(ex_out) - sum(ex_in)

-67666.27929135691

Looking at the exergy shares:

In [23]:
sum(ex_in) / (h_in - h0), sum(ex_out) / (h_out - h0)

(0.3084650109659069, 0.28404194356290713)